In [0]:
import pandas as pd

In [0]:
storageAccountName = "az866353"
storageAccountAccessKey = "J/nNU/Gaz158Z0NsYBvbcIkr+PS0KyCedNzpHk7leqLYRi9+bInU9BzJuJ2qNqgwRuz+JWfRvLvl+AStk4ls2w=="
blobContainerName = "aztest"
file_url = 'https://www.kvn.de/internet_media/Mitglieder/Verordnungen/Arzneimittel/Arzneimittelvereinbarung+2023+_+Vorabver%C3%B6ffentlichung/AMV_2023+nebst+Anlagen.pdf'

filename = "/dbfs/mnt/FileStore/aztest/Entgelttarif.pdf"

#filename = "/dbfs/mnt/FileStore/aztest/KV_Niedersachsen_AMV_2022_Medicine_Guideline.pdf"

In [0]:
if not any(mount.mountPoint == f"/mnt/FileStore/{blobContainerName}/" for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
    source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
    mount_point = f"/mnt/FileStore/{blobContainerName}/",
    extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
  )
  except Exception as e:
    print("already mounted. Try to unmount first")

In [0]:
import os

def file_exists(path):
  if path.startswith("/dbfs/"):
    path=path.replace("/dbfs/","dbfs:/")
  
  try:
    dbutils.fs.ls(path)
    return True
  except Exception as e:
    return False

      
def get_list_of_files(path):
  arr = os.listdir(path)
  return arr



def check_create_dir(path):
  if not os.path.isdir(path):
    os.mkdir(path)

In [0]:
source_folder = "/dbfs/mnt/FileStore/aztest/"

Download Rundschreiben

In [0]:
list_years = [2015, 2016, 2017, 2018, 2019, 2020,2021,2022,2023] 
#list_years = [2015, 2016] 


In [0]:
import requests
from PyPDF2 import PdfReader
import time

for year in list_years:
    url = f'https://www.kvn.de/internet_media/Mitglieder/Praxisf%C3%BChrung/Publikationen/Rundschreiben/Rundschreiben+{year}.pdf'
    response = requests.get(url)
    filename_pdf= f'{source_folder}{year}.pdf'
    
    with open(filename_pdf, 'wb') as f:
        f.write(response.content)
        

In [0]:
def get_Text_with_content(filename,search_values):
    """
    This code sample shows Prebuilt Layout operations with the Azure Form Recognizer client library. 
    The async versions of the samples require Python 3.6 or later.

    To learn more, please visit the documentation - Quickstart: Form Recognizer Python client library SDKs
    https://docs.microsoft.com/en-us/azure/applied-ai-services/form-recognizer/quickstarts/try-v3-python-sdk
    """

    from azure.core.credentials import AzureKeyCredential
    from azure.ai.formrecognizer import DocumentAnalysisClient

    """
    Remember to remove the key from your code when you're done, and never post it publicly. For production, use
    secure methods to store and access your credentials. For more information, see 
    https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-security?tabs=command-line%2Ccsharp#environment-variables-and-application-configuration
    """
    endpoint = "https://formrec866353.cognitiveservices.azure.com/"
    key = "c647473722c1402c836414afe191d81e"

    model_id="prebuilt-layout"


    with open(filename, "rb") as fd:
        document = fd.read()

    document_analysis_client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))
    poller = document_analysis_client.begin_analyze_document(model_id, document)


    # sample document
    #formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"

    #document_analysis_client = DocumentAnalysisClient(
    #    endpoint=endpoint, credential=AzureKeyCredential(key)
    #)
        
    #poller = document_analysis_client.begin_analyze_document_from_url("prebuilt-layout", formUrl)


    result = poller.result()


    list_page_number =[]
    list_of_detections=[]
    list_of_search_values_findings = []
    list_of_df = []

 
    for para in result.paragraphs:
        text= para.content
        for search_value in search_values:
            if search_value in text:
                list_of_detections.append(text)
                list_page_number.append(para.bounding_regions[0].page_number)
                list_of_search_values_findings.append(search_value)
    list_of_df.append(pd.DataFrame({'Filename': filename,'page_number': list_page_number, 'content': list_of_detections, 'search_value': list_of_search_values_findings}))

    
    concatenated_df = pd.concat(list_of_df)
    return concatenated_df
    



In [0]:
for year in list_years:
    filename_pdf= f'{source_folder}{year}.pdf'
    filename_txt =  f'{source_folder}{year}.pdf.txt'
    
    # Erstellen Sie ein PDF-Reader-Objekt
    pdf_reader = PdfReader(filename_pdf)
    # Lesen Sie den Text aus der PDF-Datei
    text = ''
    for page in pdf_reader.pages:
        text += page.extract_text()
    # Schreiben Sie den Text in eine Textdatei
    with open(filename_txt, 'w', encoding='utf-8') as f:
        f.write(text)

In [0]:

list_of_df = []
list_of_search_values =["Dapagliflozin","AstraZeneca","Empagliflozin","Böhringer","Bayer","Corona"]
for year in list_years:
    filename_pdf = source_folder+f"{year}.pdf"
 
    list_of_df.append(get_Text_with_content(filename_pdf,list_of_search_values))
    print(filename_pdf)
    


concatenated_df = pd.concat(list_of_df)
concatenated_df.head()

In [0]:
spark.sql("DROP TABLE IF EXISTS Detections")


In [0]:
from pyspark.sql import SparkSession
import pandas as pd

# Erstellen einer SparkSession
spark = SparkSession.builder.appName('pandas_to_spark').getOrCreate()

# Konvertieren des Pandas DataFrame in einen Spark DataFrame
sdf = spark.createDataFrame(concatenated_df)

# Speichern des Spark DataFrame als Tabelle
sdf.write.mode('overwrite').saveAsTable('Detections')

In [0]:
%sql
select * from Detections 